In [ ]:
!pip install transformers==2.8.0 #Installing the transformers library

In [ ]:
import os #Importing the OS module
import random #Importing the random module
import time #Importing the time module
from tqdm import tqdm #To display progress bars 
import numpy as np  
import pandas as pd
from sklearn.metrics import f1_score #Importing the f1_score metric
import requests #Import the requests module
import torch #Importing the torch library
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler, RandomSampler
from transformers import BertTokenizer #Transformer import for tokenization
from transformers import BertModel #Transformer import for the BertModel
from transformers import get_linear_schedule_with_warmup, AdamW #Optimization imports

In [ ]:
#Collecting the raw tweet data from github for the training,validation and testing files

TRAIN_IN = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt'
VAL_IN = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_text.txt'
TEST_IN = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt'

TRAIN_OUT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt'
VAL_OUT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_labels.txt'
TEST_OUT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt'

In [ ]:
#This function is used to convert and store the tweets and labels into their own dataframes
#inputs represent the tweets and outputs represents the labels
def txtdf(inputs, outputs):
    post = []
    targets = []
    for sentence in inputs.split('\n'):
        post.append(sentence)
    for target in outputs.split('\n'):
        try:
            targets.append(int(target))
        except ValueError:
            pass
    dataframe= pd.DataFrame(post[:-1], columns=['text'])
    dataframe['prediction'] = targets
   
    return dataframe

#This function is used to retrieve the contents of the different datasets so that they can be converted to a dataframe

def reqtxt (T_IN, T_OUT, V_IN, V_OUT, TE_IN, TE_OUT):
  
  ti = requests.get(T_IN).text
  to = requests.get(T_OUT).text
  tei = requests.get(TE_IN).text
  teo = requests.get(TE_OUT).text
  vi = requests.get(V_IN).text
  vo = requests.get(V_OUT).text
  training = txtdf(ti, to)
  testing = txtdf(tei, teo)  
  validation = txtdf(vi, vo)
  
  return training, testing, validation

In [ ]:
#Obtaining the dataframes for the training, validation and testing inputs / targets
training, validation, testing = reqtxt(TRAIN_IN, TRAIN_OUT,VAL_IN, VAL_OUT,TEST_IN, TEST_OUT)

In [ ]:
if torch.cuda.is_available():       #Checking for GPU availability
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
#Calling the BERT Tokenizer
token = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


def preprocessing_for_bert(data): #Function to perform the tokenization process

    ids = [] #Used to hold the id output of the tokenization
    attention_masks = [] #Used to hold the attention output of the tokenization

    for sent in data:
        encoded_sent = token.encode_plus(
            text=sent, 
            add_special_tokens=True,   #Includes tokens for separations
            max_length=MAX_LEN,        # The maximum length for truncation
            pad_to_max_length=True,    # The length to which sentences will be padded
            return_attention_mask=True) # To return the attention mask
            
        #Appending the outputs of the tokenization to the empty lists
        ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Type conversion to tensor
    ids = torch.tensor(ids)
    attention_masks = torch.tensor(attention_masks)

    return ids, attention_masks

In [ ]:
MAX_LEN = 64 #Initializing the maximum length to 64
#Retrieving the values for our training and validation inputs
token_ids = list(preprocessing_for_bert([training.text[0]])[0].squeeze().numpy())
X_train = training.text.values
X_val = testing.text.values

# Tokenization function used on the training and validation inputs to obtain the ids and attention masks
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

In [ ]:
#Converting the labels of the training and validation datasets to torch.Tensor
train_labels = torch.tensor(training.prediction.values)
val_labels = torch.tensor(testing.prediction.values)

batch_size = 16 #Batch size set at 16 as it gives the best F1-Score

# We create the dataloader for training here
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# We create the dataloader for validation here
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
# Creating a class for the Bert Classifier
class BertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):

        super(BertClassifier, self).__init__()
        D_in, H, D_out = 768, 50, 2 #D_out is the number of targets we need for our dataset
        self.bert = BertModel.from_pretrained('bert-base-uncased') #Creating an instance of BERT
        #Creating an instance of the feedforward classifier
        self.classifier = nn.Sequential(nn.Linear(D_in, H),nn.ReLU(),nn.Linear(H, D_out))

        if freeze_bert: #Freezing BERT
            for param in self.bert.parameters():
                param.requires_grad = False

# Function to feed the tokens (IDs and Attention masks) to BERT

    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        last_hidden_state_cls = outputs[0][:, 0, :] #Using last cls for classification
        logits = self.classifier(last_hidden_state_cls) #Computing logits

        return logits

In [ ]:
#Function to initialize the BERT model
def initialize_model(epochs=4):
    bert_classifier = BertClassifier(freeze_bert=False) #Creating an instance of the BERT classifier
    bert_classifier.to(device) #PyTorch to run the model on GPU if available

    # Creating the optimizer
    # Defining learning rate of the model 
    # Defining Epsilon value of the model 
    optimizer = AdamW(bert_classifier.parameters(),lr=5e-5,eps=1e-8)
    
    #No. of steps required for training
    total_steps = len(train_dataloader) * epochs

    # Defining the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
#Defining the loss function for the model
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
 
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

#Function used for training of the model
def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):

    for epoch_i in range(epochs):
        print(f"{'Epoch':^6} | {'Batch':^6} | {'Train Loss':^10} | {'Val Loss':^8} | {'Val Acc':^6} | {'Elapsed':^6}")
        #Calculating time elapsed for each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Re-initialising the variables for next iteration
        total_loss, batch_loss, batch_counts = 0, 0, 0

        model.train()

        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Send batch to device's processor
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Setting gradients back to zero
            model.zero_grad()

            # Feedfoward pass of the model
            logits = model(b_input_ids, b_attn_mask)

            # Calculating the loss values and aggregating them
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Backpropagation pass to calculate gradients
            loss.backward()

            # Clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Updating of the hyperparameters as defined in optimizer
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 50 batches
            if (step % 50 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                #Calculate the elapsed time for 50 batches
                time_elapsed = time.time() - t0_batch
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reseting the batch track variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Computing the average training loss 
        avg_train_loss = total_loss / len(train_dataloader)
        print("****************************************")

        #Assessing the model's performance on the validation set
        if evaluation == True:
            val_loss, val_accuracy = evaluate(model, val_dataloader)
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
        print("\n")
    
    print("Model was run successfully.")

def evaluate(model, val_dataloader):

    model.eval() #model in evaluation mode

    #Creating tracking variables
    val_accuracy = []
    val_loss = []

    for batch in val_dataloader:
        # Sending the batch to the device's processor
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Computing the logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Computing the loss value of the model
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Obtaining the predictions of the model
        preds = torch.argmax(logits, dim=1).flatten()

        # Obtaining the f1 score for each classification task
        accuracy = f1_score(b_labels.tolist(), preds.tolist(), average = 'macro' )
        val_accuracy.append(accuracy)

    # Computing overall performance of the BERT classifier
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
#Running the model
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)